# Sprint 10. Tasca 01. Aprendizaje supervisado. Clasificación
## By José Manuel Castaño

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix, recall_score, roc_auc_score
from imblearn.metrics import specificity_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.neural_network import MLPClassifier

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
airlines = pd.read_csv('DelayedFlights.csv', index_col=0, engine='python')

In [4]:
airlines

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,N726SW,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,2008,12,13,6,1250.0,1220,1617.0,1552,DL,1621,N938DL,147.0,152.0,120.0,25.0,30.0,MSP,ATL,906,9.0,18.0,0,N,0,3.0,0.0,0.0,0.0,22.0
7009717,2008,12,13,6,657.0,600,904.0,749,DL,1631,N3743H,127.0,109.0,78.0,75.0,57.0,RIC,ATL,481,15.0,34.0,0,N,0,0.0,57.0,18.0,0.0,0.0
7009718,2008,12,13,6,1007.0,847,1149.0,1010,DL,1631,N909DA,162.0,143.0,122.0,99.0,80.0,ATL,IAH,689,8.0,32.0,0,N,0,1.0,0.0,19.0,0.0,79.0
7009726,2008,12,13,6,1251.0,1240,1446.0,1437,DL,1639,N646DL,115.0,117.0,89.0,9.0,11.0,IAD,ATL,533,13.0,13.0,0,N,0,NaN,NaN,NaN,NaN,NaN


La descripción de los diferentes campos, el estudio de la distribución de las variables, los Nans, etc, ya lo hemos realizado en exercicios anteriores. 

Elimimamos del data set los vuelo cancelados y los desviados ya que no nos sirven para estudiar los retrasos. A continuación, eliminamos los campos **Cancelled, CancellationCode, Diverted**

In [3]:
airlines=airlines[airlines['Cancelled']==0]
airlines=airlines[airlines['Diverted'] ==0]

### Eliminación de variables

En este primer modelo, eliminamos algunas variables por diferentes motivos:
    
- **Cancelled, CancellationCode, Diverted** porque no sirven para estudiar los retrasos
- **CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, SecurityDelay, LateAircraftDelay** ya que explican la causa del retraso para los vuelos con retraso >15' de forma exacta. Quizás se han registrado cuando ya se conoce el retraso y se han cuadrado exactamente con el retraso.
- **ActualElapsedTime** ya que ActualElapsedTime = AirTime + TaxiIn + TaxiOut
- **FlightNum, TailNum** perque entendemos que no afectan al target.
- **Year** perque siempre es 2008
- **DepTime, ArrTime, CRSArrTime, AirTime, CRSElapsedTime**. Nos quedemos CRSDepTime como variable de horario

En función de los resultados de los modelos, quizás volveríamos a recuperar algunas de las variables inicialmente rechazadas.


In [4]:
airlines1=airlines.drop(['Cancelled', 'CancellationCode', 'Diverted','CarrierDelay','WeatherDelay', 'NASDelay', 'SecurityDelay', 'SecurityDelay', 'LateAircraftDelay','ActualElapsedTime','FlightNum','TailNum', 'Year','DepTime', 'ArrTime', 'CRSArrTime', 'AirTime', 'CRSElapsedTime'], axis=1)
airlines1

,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut
0,1,3,4,1955,WN,-14.0,8.0,IAD,TPA,810,4.0,8.0
1,1,3,4,735,WN,2.0,19.0,IAD,TPA,810,5.0,10.0
2,1,3,4,620,WN,14.0,8.0,IND,BWI,515,3.0,17.0
4,1,3,4,1755,WN,34.0,34.0,IND,BWI,515,3.0,10.0
5,1,3,4,1915,WN,11.0,25.0,IND,JAX,688,4.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,1220,DL,25.0,30.0,MSP,ATL,906,9.0,18.0
7009717,12,13,6,600,DL,75.0,57.0,RIC,ATL,481,15.0,34.0
7009718,12,13,6,847,DL,99.0,80.0,ATL,IAH,689,8.0,32.0
7009726,12,13,6,1240,DL,9.0,11.0,IAD,ATL,533,13.0,13.0


## - Exercici 1

Crea almenys tres models de classificació diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv. Considera si el vol ha arribat tard o no (ArrDelay > 0).

Creamos un nuevo campo **Target** con los siguientes valores:   
- 0   si ArrDelay<= 0
- 1   si ArrDelay >0

In [5]:
airlines1['Target'] = airlines1['ArrDelay']>0
airlines1

,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Target
0,1,3,4,1955,WN,-14.0,8.0,IAD,TPA,810,4.0,8.0,False
1,1,3,4,735,WN,2.0,19.0,IAD,TPA,810,5.0,10.0,True
2,1,3,4,620,WN,14.0,8.0,IND,BWI,515,3.0,17.0,True
4,1,3,4,1755,WN,34.0,34.0,IND,BWI,515,3.0,10.0,True
5,1,3,4,1915,WN,11.0,25.0,IND,JAX,688,4.0,10.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009710,12,13,6,1220,DL,25.0,30.0,MSP,ATL,906,9.0,18.0,True
7009717,12,13,6,600,DL,75.0,57.0,RIC,ATL,481,15.0,34.0,True
7009718,12,13,6,847,DL,99.0,80.0,ATL,IAH,689,8.0,32.0,True
7009726,12,13,6,1240,DL,9.0,11.0,IAD,ATL,533,13.0,13.0,True


In [6]:
#Vemos la distribución del Target
airlines1.Target.value_counts(normalize=True)

True     0.893715
False    0.106285
Name: Target, dtype: float64

Observamos que se trata de un dataset **desbalanceado**, donde el **89,4% de los vuelos llegan retrasados** y el **10.6% llegan a tiempo**.
Inicialmente entrenaremos los modelos con una muestra aleatoria y posteriormente veremos la influencia en el modelo cuando utilizamos datos más balanceados.

Creamos la función **obtener_metricas(modelo, y_true, y_pred)** que a partir de un modelo entrenado retorna sus métricas

In [19]:
def obtener_metricas(modelo, y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)
    specificity = specificity_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)
    rocaucscore= roc_auc_score(y_true, y_pred)
    return {'Modelo': modelo, 'Accuracy':accuracy, 'Sensitivity':sensitivity, 'Specificity': specificity, 'Precision':precision, 'F1Score':f1score, 'ROC AUC Score':rocaucscore}


Sabemos que la variable **DepDelay** tiene una gran influencia en el target. Para determinarlo, simulamos un modelo el cual devuelve la propia variable transformada (1 si DepDelay tiene retraso, 0 si no lo tiene). Utilizamos todo el dataset

In [7]:
# Utilizando únicamente la variable DepDelay transformándola en 0/1
y_pred = airlines1['DepDelay']>0
obtener_metricas('prueba', airlines1.Target, y_pred)

{'Modelo': 'prueba',
 'Accuracy': 0.8937154728006177,
 'Sensitivity': 1.0,
 'Specificity': 0.0,
 'Precision': 0.8937154728006177,
 'F1Score': 0.9438751339755397,
 'ROC AUC Score': 0.5}

In [9]:
confusion_matrix(airlines1.Target, y_pred)

array([[      0,  204956],
       [      0, 1723415]], dtype=int64)

Observamos que con un modelo que simplemente retorne DepDelay obtenemos una Accuracy y precision de 0.89, una sensitivity de 1 y F1score de 0.94. Por contra vemos que no ha detectado ningún negativo y tiene 204956 false positive  

Por motivos de capacidad de procesado tomamos una muestra aleatoria del 10% y será con la que realizaremos los modelos

In [7]:
airlines1 = airlines1.sample(frac=0.1, random_state=0)

Hacemos una ingeniería de variables inicial:
- Transformaremos **CRSDepTime** en intervalos horarios
- Dummificaremos **UniqueCarrier**

In [8]:
airlines1['CRSDepTime'] = airlines1['CRSDepTime'] //100
airlines1= pd.get_dummies(airlines1, columns=['UniqueCarrier'])

In [9]:
airlines1

,Month,DayofMonth,DayOfWeek,CRSDepTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Target,UniqueCarrier_9E,UniqueCarrier_AA,UniqueCarrier_AQ,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
6731208,12,1,1,15,84.0,94.0,HOU,HRL,276,2.0,8.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
609057,2,3,7,10,7.0,18.0,SJC,SEA,697,5.0,7.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
889032,2,2,6,19,1.0,8.0,LAS,ONT,197,4.0,10.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5502092,10,2,4,18,31.0,35.0,IAH,JAX,817,5.0,18.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1699584,3,25,2,18,62.0,66.0,DFW,PBI,1103,5.0,12.0,True,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511846,1,13,7,19,46.0,35.0,AUS,DFW,190,15.0,17.0,True,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
636696,2,12,2,21,81.0,56.0,ISP,MCO,972,5.0,24.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5079318,9,6,6,18,254.0,268.0,PHL,PHX,2075,6.0,15.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
51952,1,18,5,13,92.0,106.0,TPA,MDW,997,6.0,8.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Seleccionaremos las variables: **Month, DayofMonth, DayOfWeek, CRSDepTime,'DepDelay', 'Distance', 'TaxiIn', 'TaxiOut' y UniqueCarrier** dummificada

Para entrenar y testear los diferentes modelos generamos 2 datasets:
- Train/Test aleatorio de la muestra (**no balanceado**)
- Train/Test **balanceado** (al 30%) de la muestra

In [10]:
#Datos no balanceados
y= airlines1['Target'].to_numpy()
x= airlines1.drop(['ArrDelay', 'Target', 'Origin', 'Dest'], axis=1).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

Para obtener otro dataset de train más balanceado realizamos:   
- Smote para aumentar la clase minoritaria y pase de representar el 10% al representar aprox. un 17% del total    
- Undersampling para disminuir la clase mayoritaria y dejar aproximadamente un **30% en la clase minoritaria**

In [11]:
#Aplicamos smote 
X_train_smote, y_train_smote = SMOTE(sampling_strategy= 0.20, random_state=0).fit_resample(X_train,y_train)

In [12]:
y_train_smote.mean()

0.833335634691595

In [13]:
#Hacemos un undersampling
X_train_b ,y_train_b = RandomUnderSampler(sampling_strategy= 0.41, random_state=0).fit_resample(X_train_smote, y_train_smote)

In [14]:
y_train_b.mean()

0.7092196872967308

In [15]:
y_train_b.shape

(83018,)

In [16]:
y_train.shape

(134985,)

Tenemos un dataset de train más balanceado (al 30%) pero con menos datos. Hemos pasado de 134985 a 83018 registros

 ### Model 1: Logistic Regression

In [17]:
#Logistic Regression datos no balanceados
lr = LogisticRegression(max_iter=1000,random_state=0)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [20]:
metricas= obtener_metricas('LogisticRegression', y_test, lr.predict(X_test))
metricas

{'Modelo': 'LogisticRegression',
 'Accuracy': 0.9026308511373851,
 'Sensitivity': 0.9808335750894498,
 'Specificity': 0.24483487880266797,
 'Precision': 0.9161427848833008,
 'F1Score': 0.9473851355769141,
 'ROC AUC Score': 0.6128342269460589}

In [21]:
#Generamos un DF con las métricas para poder hacer una comparativa
metricas1 = pd.DataFrame(metricas, index=[0])
metricas1

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression,0.902631,0.980834,0.244835,0.916143,0.947385,0.612834


### Model 1.b: Logistic Regression (datos balanceados)

In [22]:
#Logistic Regression datos balanceados al 30%
lr_b = LogisticRegression(max_iter=1100, random_state=0)
lr_b.fit(X_train_b, y_train_b)

LogisticRegression(max_iter=1100, random_state=0)

In [23]:
metricas= obtener_metricas('LogisticRegression_balanced', y_test, lr_b.predict(X_test))
metricas

{'Modelo': 'LogisticRegression_balanced',
 'Accuracy': 0.8654670538615779,
 'Sensitivity': 0.8804757760371338,
 'Specificity': 0.7392223849032048,
 'Precision': 0.9659862502121881,
 'F1Score': 0.9212509991601996,
 'ROC AUC Score': 0.8098490804701692}

In [24]:
#Añadimos las métricas al DF metricas1 para la comparativa posterior
metricas1 = metricas1.append(metricas, ignore_index=True)

### 2.- Ada Boost Classifier

In [25]:
#Ada Bost Classifier con datos no balanceados
adab = AdaBoostClassifier(random_state=0)
adab.fit(X_train, y_train)

AdaBoostClassifier(random_state=0)

In [26]:
metricas= obtener_metricas('AdaBoostClassifier', y_test, adab.predict(X_test))
metricas

{'Modelo': 'AdaBoostClassifier',
 'Accuracy': 0.9017665767821337,
 'Sensitivity': 0.9741997872546175,
 'Specificity': 0.2925004067024565,
 'Precision': 0.9205226608187135,
 'F1Score': 0.9466008926474042,
 'ROC AUC Score': 0.6333500969785371}

In [27]:
#Añadimos las métricas al DF metricas1 para la comparativa posterior
metricas1 = metricas1.append(metricas, ignore_index=True)

### 2.b- Ada Boost Classifier (datos balanceados) 

In [28]:
#Ada Bost Classifier con datos balanceados al 30%
adab_b = AdaBoostClassifier(random_state=0)
adab_b.fit(X_train_b, y_train_b)

AdaBoostClassifier(random_state=0)

In [29]:
metricas= obtener_metricas('AdaBoostClassifier_balanced', y_test, adab_b.predict(X_test))
metricas

{'Modelo': 'AdaBoostClassifier_balanced',
 'Accuracy': 0.8739887990043559,
 'Sensitivity': 0.8986945169712793,
 'Specificity': 0.6661786237188873,
 'Precision': 0.9577072899276572,
 'F1Score': 0.9272629310344828,
 'ROC AUC Score': 0.7824365703450833}

In [30]:
#Añadimos las métricas al DF metricas1 para la comparativa posterior
metricas1 = metricas1.append(metricas, ignore_index=True)

### 3.- BalancedBaggingClassifier

Utilizamos un modelo preparado para datasets no balanceados, ya que incorpora un balanceo previo de los datos

In [31]:
#BalancedBaggingClassifier con datos no balanceados
bbc = BalancedBaggingClassifier(random_state=0)
bbc.fit(X_train, y_train)

BalancedBaggingClassifier(random_state=0)

In [32]:
metricas= obtener_metricas('BalancedBaggingClassifier', y_test, bbc.predict(X_test))
metricas

{'Modelo': 'BalancedBaggingClassifier',
 'Accuracy': 0.811743759939155,
 'Sensitivity': 0.8075621313219224,
 'Specificity': 0.8469171953798601,
 'Precision': 0.9779604646805321,
 'F1Score': 0.8846304594230994,
 'ROC AUC Score': 0.8272396633508914}

In [33]:
#Añadimos las métricas al DF metricas1 para la comparativa posterior
metricas1 = metricas1.append(metricas, ignore_index=True)

## - Exercici 2

Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.

Realizamos la comparativa de los 3 modelos y sus variantes con datos balanceados y marcamos el valor máximo en cada una de las métricas

In [34]:
metricas1.style.highlight_max(['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1Score', 'ROC AUC Score'])

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression,0.902631,0.980834,0.244835,0.916143,0.947385,0.612834
1,LogisticRegression_balanced,0.865467,0.880476,0.739222,0.965986,0.921251,0.809849
2,AdaBoostClassifier,0.901767,0.974200,0.292500,0.920523,0.946601,0.633350
3,AdaBoostClassifier_balanced,0.873989,0.898695,0.666179,0.957707,0.927263,0.782437
4,BalancedBaggingClassifier,0.811744,0.807562,0.846917,0.977960,0.884630,0.827240


De la comparación de los modelos con datos sin balancear y datos más balanceados, tenemos:
- Los modelos no balanceados, aunque presentan una accuracy, sensitivity y precision altas, tienen en realidad un pobre desempeño ya que la specificity es muy baja (menor de 0.3)
- Al balancear más los datos, los modelos bajan ligeramente en accuracy (4%) i en sensitivity (9%), Por contra aumentan sustancialmente en specificity (más del doble), en ROC AUC Score (25%) y en precision (4%)
- El modelo **BalancedBaggingClassifier** que previamente balancea los datos presenta las mejores metricas de Specificity, Precision y Roc AUC Score por contra presenta una Accuracy y una Specificity que son las más bajas de todos los modelos.  

En cuanto a las mejores métricas presentadas por cada uno de los modelos, tenemos:    
  
- Accuracy: El modelo **Logistic Regression** presenta el mejor dato con 0.903
- Sensitivity: El modelo **Logistic Regression** presenta el mejor dato con 0.981
- Specificity: El modelo **BalancedBaggingClassifier** presenta el mejor dato con 0.847
- Precision: El modelo **BalancedBaggingClassifier** presenta el mejor dato con 0.978
- F1Score: El modelo **Logistic Regression** presenta el mejor dato con 0.947
- ROC AUC Score: El modelo **BalancedBaggingClassifier** presenta el mejor dato con 0.827

Analizamos las matrices de confusión

In [35]:
print('Logistic Regression no balanced')
print(confusion_matrix(y_test, lr.predict(X_test)))
print()
print('Logistic Regression balanced')
print(confusion_matrix(y_test, lr_b.predict(X_test)))
print()
print('Ada Boost Classifier no balanced')
print(confusion_matrix(y_test, adab.predict(X_test)))
print()
print('Ada Boost Classifier balanced')
print(confusion_matrix(y_test, adab_b.predict(X_test)))
print()
print('BalancedBaggingClassifier')
print(confusion_matrix(y_test, bbc.predict(X_test)))
print()

Logistic Regression no balanced
[[ 1505  4642]
 [  991 50714]]

Logistic Regression balanced
[[ 4544  1603]
 [ 6180 45525]]

Ada Boost Classifier no balanced
[[ 1798  4349]
 [ 1334 50371]]

Ada Boost Classifier balanced
[[ 4095  2052]
 [ 5238 46467]]

BalancedBaggingClassifier
[[ 5206   941]
 [ 9950 41755]]



Observamos:
- Cuando utilizamos datos balanceados en Logistic Regression y en Ada Boost Classifier, aumentamos casi el triple la detección de negativos y disminuimos los falsos positivos a menos de la mitad. Por contra aumentamos más del triple los falsos negativos.
- Balanced Bagging detecta el mayor número de negativos pero también presenta un número muy elvado de falsos negativos y no es tan bueno detectando los positivos.
- Ada Boost Classifier no balanced detecta el mayor número de aciertos y un número de falsos negativos bajo, por contra presenta un número elevado de falsos positivos
- Logistic Regression no balanced detecta el mayor número de positivos

Depende de las métricas que queramos optimizar o de las implicaciones que puedan tener las detecciones o las falsas detecciones, eligiríamos el algoritmo.

## - Exercici 3

Entrena’ls utilitzant els diferents paràmetres que admeten.

Para determinar los mejores parámetros utilizaremos GridSearch. Buscamos mejorar el **ROC AUC Score**

 ### Model 1: Logistic Regression

In [76]:
#Vemos los parámetros por defecto
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [34]:
# GridSearch
parameters ={'class_weight': ['Balanced', 'auto', None], 'solver': ['newton-cg', 'lbfgs'], 'max_iter': [1000]}
gs = GridSearchCV(estimator = LogisticRegression(), param_grid = parameters, scoring = 'roc_auc')
gs.fit(X_train, y_train)

print("Best parameters:", gs.best_params_)

Best parameters: {'class_weight': 'Balanced', 'max_iter': 1000, 'solver': 'newton-cg'}


Entrenamos la Logistic Regression con los parámetros propuestos

In [36]:
lr_gs = LogisticRegression(class_weight= 'Balanced', max_iter= 1000, solver= 'newton-cg', random_state=0)
lr_gs.fit(X_train, y_train)

LogisticRegression(class_weight='Balanced', max_iter=1000, random_state=0,
                   solver='newton-cg')

In [37]:
metricas= obtener_metricas('LogisticRegression_optim', y_test, lr_gs.predict(X_test))
metricas

{'Modelo': 'LogisticRegression_optim',
 'Accuracy': 0.9028037060084353,
 'Sensitivity': 0.9807562131321922,
 'Specificity': 0.24711241255897184,
 'Precision': 0.9163684990422495,
 'F1Score': 0.947469708434928,
 'ROC AUC Score': 0.6139343128455821}

In [38]:
#Generamos un DF con las métricas para poder hacer una comparativa
metricas2 = pd.DataFrame(metricas, index=[0])
metricas2

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression_optim,0.902804,0.980756,0.247112,0.916368,0.94747,0.613934


### 2.- Ada Boost Classifier

In [38]:
#Vemos los parámetros por defecto
adab.get_params()

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 50,
 'random_state': 0}

In [39]:
# GridSearch
parameters ={'n_estimators': [30, 50, 70, 90]}
gs = GridSearchCV(estimator = AdaBoostClassifier(), param_grid=parameters, scoring = 'roc_auc')
gs.fit(X_train, y_train)

print("Best parameters:", gs.best_params_)

Best parameters: {'n_estimators': 90}


Entrenamos Ada Bost Classifier con los parámetros propuestos

In [39]:
adab_gs = AdaBoostClassifier(n_estimators= 90, random_state=0)
adab_gs.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=90, random_state=0)

In [40]:
metricas= obtener_metricas('AdaBoostClassifier_optim', y_test, adab_gs.predict(X_test))
metricas

{'Modelo': 'AdaBoostClassifier_optim',
 'Accuracy': 0.9028728479568554,
 'Sensitivity': 0.9747413209554202,
 'Specificity': 0.2983569220758093,
 'Precision': 0.9211690305600234,
 'F1Score': 0.947198285988141,
 'ROC AUC Score': 0.6365491215156148}

In [41]:
#Añadimos las métricas al DF metricas2 para la comparativa posterior
metricas2 = metricas2.append(metricas, ignore_index=True)

### 3.- BalancedBaggingClassifier

In [43]:
#Vemos los parámetros por defecto
bbc.get_params()

{'base_estimator': None,
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'replacement': False,
 'sampler': None,
 'sampling_strategy': 'auto',
 'verbose': 0,
 'warm_start': False}

In [44]:
# GridSearch
parameters ={'n_estimators': [10, 20, 30]}
gs = GridSearchCV(estimator = BalancedBaggingClassifier(), param_grid=parameters, scoring = 'roc_auc')
gs.fit(X_train, y_train)

print("Best parameters:", gs.best_params_)

Best parameters: {'n_estimators': 30}


Entrenamos BalancedBaggingClassifier con los parámetros propuestos

In [42]:
bbc_gs = BalancedBaggingClassifier(n_estimators= 30, random_state=0)
bbc_gs.fit(X_train, y_train)

BalancedBaggingClassifier(n_estimators=30, random_state=0)

In [43]:
metricas= obtener_metricas('BalancedBaggingClassifier_optim', y_test, bbc_gs.predict(X_test))
metricas

{'Modelo': 'BalancedBaggingClassifier_optim',
 'Accuracy': 0.8251745834197608,
 'Sensitivity': 0.8226477129871386,
 'Specificity': 0.8464291524320807,
 'Precision': 0.9782883690977253,
 'F1Score': 0.8937426458228274,
 'ROC AUC Score': 0.8345384327096097}

In [44]:
#Añadimos las métricas al DF metricas2 para la comparativa posterior
metricas2 = metricas2.append(metricas, ignore_index=True)

Comparamos las métricas de los modelos iniciales con las métricas de los modelos optimizados

In [81]:
pd.concat([metricas1.loc[[0,2,4]],metricas2 ], axis=0)

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression,0.902631,0.980834,0.244835,0.916143,0.947385,0.612834
2,AdaBoostClassifier,0.901767,0.974200,0.292500,0.920523,0.946601,0.633350
4,BalancedBaggingClassifier,0.811744,0.807562,0.846917,0.977960,0.884630,0.827240
0,LogisticRegression_optim,0.902804,0.980756,0.247112,0.916368,0.947470,0.613934
1,AdaBoostClassifier_optim,0.902873,0.974741,0.298357,0.921169,0.947198,0.636549
2,BalancedBaggingClassifier_optim,0.825175,0.822648,0.846429,0.978288,0.893743,0.834538


Al comparar las métricas de los modelos iniciales con las de los modelos con parámetros optimizados observamos que se produce una **mejora muy poco significativa** en todas las métricas, en el mejor de los casos del 1,4%.

### - Exercici 4

Compara el seu rendiment utilitzant l’aproximació traint/test o cross-validation.

Vamos a realizar un **Cross Validation Score** de los modelos optimizados para la métrica **F1**. Compararemos los resultados con los de Train/Test para esos mismos modelos

In [66]:
#Cross Validation Score para Logistic Regression
lr_cv = LogisticRegression(class_weight= 'Balanced', max_iter= 1000, solver= 'newton-cg', random_state=0)
f1 = cross_val_score(lr_cv, x, y, cv = 10, scoring = 'f1')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


In [70]:
print('f1 Score: {0}     std_F1 Score: {1}'.format(np.mean(f1).round(4), np.std(f1).round(4)))

f1 Score: 0.9475     std_F1 Score: 0.0005


In [71]:
#Cross Validation Score para Ada Boost Classifier
adab_cv = AdaBoostClassifier(n_estimators= 90, random_state=0)
f1 = cross_val_score(adab_cv, x, y, cv = 10, scoring = 'f1')

In [72]:
print('f1 Score: {0}     std_F1 Score: {1}'.format(np.mean(f1).round(4), np.std(f1).round(4)))

f1 Score: 0.9474     std_F1 Score: 0.0004


In [73]:
#Cross Validation Score para BalancedBaggingClassifier
bbc_cv = BalancedBaggingClassifier(n_estimators= 30, random_state=0)
f1 = cross_val_score(bbc_cv, x, y, cv = 10, scoring = 'f1')

In [74]:
print('f1 Score: {0}     std_F1 Score: {1}'.format(np.mean(f1).round(4), np.std(f1).round(4)))

f1 Score: 0.8953     std_F1 Score: 0.0014


Observamos que la **desviación stándard** de f1 en todos los modelos cuando aplicamos el Cross Validation **es muy baja**. Damos por buenos los valores de f1. 

Comparamos los resultados del Train/Test y del Cross Validation para los modelos optimizados

In [83]:
metricas2['F1Score_CV'] = [0.9475, 0.9474, 0.8953]

In [84]:
metricas2[['Modelo', 'F1Score', 'F1Score_CV']]  #Modelos optimizados y entrenados con Train/Test

,Modelo,F1Score,F1Score_CV
0,LogisticRegression_optim,0.947470,0.9475
1,AdaBoostClassifier_optim,0.947198,0.9474
2,BalancedBaggingClassifier_optim,0.893743,0.8953


Hemos entrenado los modelos optimizados con el set Train/Test i con Cross Validation i **no observamos diferencias sensibles** en cuanto a la métrica **F1 Score**. La muestra de Train/Test es suficientemente amplia para entrenar adecuadamente los modelos

### - Exercici 5

Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

Realizamos las siguiente ingeniería de variables:
- RobustScaler a las variables **DepDelay, Distance, TaxiIn, TaxiOut** debido a que presentan un gran número de outliers
- Normalizamos **Month, DayofMonth, DayOfWeek, CRSDepTime**
- La variable **UniqueCarrier** ya la hemos dummificado anteriormente.

In [101]:
#Apliquem RobustScaler y MinMaxScaler
airlines2 = airlines1.copy()
columns_robust= ['DepDelay', 'Distance', 'TaxiIn', 'TaxiOut']
columns_norm= ['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime']

airlines2[columns_robust] = RobustScaler().fit_transform(airlines2[columns_robust])
airlines2[columns_norm] = MinMaxScaler().fit_transform(airlines2[columns_norm])

In [102]:
airlines2

,Month,DayofMonth,DayOfWeek,CRSDepTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Target,UniqueCarrier_9E,UniqueCarrier_AA,UniqueCarrier_AQ,UniqueCarrier_AS,UniqueCarrier_B6,UniqueCarrier_CO,UniqueCarrier_DL,UniqueCarrier_EV,UniqueCarrier_F9,UniqueCarrier_FL,UniqueCarrier_HA,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
6731208,1.000000,0.000000,0.000000,0.652174,84.0,1.666667,HOU,HRL,-0.501515,-1.00,-0.545455,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
609057,0.090909,0.066667,1.000000,0.434783,7.0,-0.142857,SJC,SEA,0.136364,-0.25,-0.636364,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
889032,0.090909,0.033333,0.833333,0.826087,1.0,-0.380952,LAS,ONT,-0.621212,-0.50,-0.363636,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5502092,0.818182,0.033333,0.500000,0.782609,31.0,0.261905,IAH,JAX,0.318182,-0.25,0.363636,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1699584,0.181818,0.800000,0.166667,0.782609,62.0,1.000000,DFW,PBI,0.751515,-0.25,-0.181818,True,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511846,0.000000,0.400000,1.000000,0.826087,46.0,0.261905,AUS,DFW,-0.631818,2.25,0.272727,True,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
636696,0.090909,0.366667,0.166667,0.913043,81.0,0.761905,ISP,MCO,0.553030,-0.25,0.909091,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5079318,0.727273,0.166667,0.833333,0.782609,254.0,5.809524,PHL,PHX,2.224242,0.00,0.090909,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
51952,0.000000,0.566667,0.666667,0.565217,92.0,1.952381,TPA,MDW,0.590909,0.00,-0.545455,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Entrenaremos los 3 modelos con los parámetros optimizados y con los datos ya escalados. Haremos un Train/Test al 30%

In [103]:
#Train/test con datos escalados y no balanceados
y= airlines2['Target'].to_numpy()
x= airlines2.drop(['ArrDelay', 'Target', 'Origin', 'Dest'], axis=1).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

### Model 1: Logistic Regression

In [104]:
#Logistic Regression con datos escalados y no balancedados
lr_esc = LogisticRegression(class_weight= 'Balanced', max_iter= 1000, solver= 'newton-cg', random_state=0)
lr_esc.fit(X_train, y_train)

LogisticRegression(class_weight='Balanced', max_iter=1000, random_state=0,
                   solver='newton-cg')

In [105]:
metricas= obtener_metricas('LogisticRegression_esc', y_test, lr_esc.predict(X_test))
metricas

{'Modelo': 'LogisticRegression_esc',
 'Accuracy': 0.9026999930858052,
 'Sensitivity': 0.9809109370467073,
 'Specificity': 0.24483487880266797,
 'Precision': 0.9161488439306359,
 'F1Score': 0.947424461775557,
 'ROC AUC Score': 0.6128729079246876}

In [106]:
#Generamos un DF con las métricas para poder hacer una comparativa
metricas2_sc = pd.DataFrame(metricas, index=[0])
metricas2_sc

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression_esc,0.9027,0.980911,0.244835,0.916149,0.947424,0.612873


### Model 2: Ada Boost Classifier

In [107]:
#Ada Bost Classifier con datos escalados y no balancedados
adab_sc = AdaBoostClassifier(n_estimators= 90, random_state=0)
adab_sc.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=90, random_state=0)

In [108]:
metricas= obtener_metricas('AdaBostClassifier_esc', y_test, adab_sc.predict(X_test))
metricas

{'Modelo': 'AdaBostClassifier_esc',
 'Accuracy': 0.9028728479568554,
 'Sensitivity': 0.9747413209554202,
 'Specificity': 0.2983569220758093,
 'Precision': 0.9211690305600234,
 'F1Score': 0.947198285988141,
 'ROC AUC Score': 0.6365491215156148}

In [109]:
#Añadimos las métricas al DF metricas2_sc para la comparativa posterior
metricas2_sc = metricas2_sc.append(metricas, ignore_index=True)

### Model 3: BalancedBaggingClassifier

In [110]:
#BalancedBaggingClassifier con datos escalados y no balancedados
bbc_sc = BalancedBaggingClassifier(n_estimators= 30, random_state=0)
bbc_sc.fit(X_train, y_train)

BalancedBaggingClassifier(n_estimators=30, random_state=0)

In [111]:
metricas= obtener_metricas('BalancedBaggingClassifier_sc', y_test, bbc_sc.predict(X_test))
metricas

{'Modelo': 'BalancedBaggingClassifier_sc',
 'Accuracy': 0.8251572979326558,
 'Sensitivity': 0.8227830964123393,
 'Specificity': 0.8451277045713356,
 'Precision': 0.9781119234836989,
 'F1Score': 0.8937488839168478,
 'ROC AUC Score': 0.8339554004918374}

In [112]:
#Añadimos las métricas al DF metricas2_sc para la comparativa posterior
metricas2_sc = metricas2_sc.append(metricas, ignore_index=True)

Comparamos las métricas de los modelos optimizados con las métricas de los modelos optimizados y datos escalados 

In [119]:
pd.concat([metricas2, metricas2_sc], axis=0).drop('F1Score_CV', axis=1)

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression_optim,0.902804,0.980756,0.247112,0.916368,0.947470,0.613934
1,AdaBoostClassifier_optim,0.902873,0.974741,0.298357,0.921169,0.947198,0.636549
2,BalancedBaggingClassifier_optim,0.825175,0.822648,0.846429,0.978288,0.893743,0.834538
0,LogisticRegression_esc,0.902700,0.980911,0.244835,0.916149,0.947424,0.612873
1,AdaBostClassifier_esc,0.902873,0.974741,0.298357,0.921169,0.947198,0.636549
2,BalancedBaggingClassifier_sc,0.825157,0.822783,0.845128,0.978112,0.893749,0.833955


Al comparar las métricas de los modelos optimizados con datos no escalados y con datos escalados observamos que no muestran diferencias sensibles en las métricas, es decir, que en estos modelos **el escalado no afecta al resultado**

### - Exercici 6

No utilitzis la variable DepDelay a l’hora de fer prediccions

A partir de los datos ya escalados del ejercicio anterior, eliminamos **DepDelay**.    

No utilizo **CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, SecurityDelay, LateAircraftDelay** ya que para los vuelos con un retraso >15', su suma es exactamente DepDelay. Si elimino DepDelay en el momento de hacer predicciones y utilizo unas variables que equivalen a su suma, poco cambiará la predicción.

In [120]:
#Train/test con datos escalados y sin la variable DepDelay
y= airlines2['Target'].to_numpy()
x= airlines2.drop(['ArrDelay', 'Target', 'Origin', 'Dest', 'DepDelay'], axis=1).to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=0)

### Model 1: Logistic Regression

In [121]:
#Logistic Regression con datos escalados y sin la variable DepDelay
lr_3 = LogisticRegression(class_weight= 'Balanced', max_iter= 1000, solver= 'newton-cg',random_state=0)
lr_3.fit(X_train, y_train)

LogisticRegression(class_weight='Balanced', max_iter=1000, random_state=0,
                   solver='newton-cg')

In [122]:
metricas= obtener_metricas('LogisticRegression_3', y_test, lr_3.predict(X_test))
metricas

{'Modelo': 'LogisticRegression_3',
 'Accuracy': 0.8930546912812003,
 'Sensitivity': 0.998317377429649,
 'Specificity': 0.007646006181877338,
 'Precision': 0.8943137322845559,
 'F1Score': 0.9434579567367006,
 'ROC AUC Score': 0.5029816918057632}

In [123]:
#Generamos un DF con las métricas para poder hacer una comparativa
metricas3 = pd.DataFrame(metricas, index=[0])
metricas3

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression_3,0.893055,0.998317,0.007646,0.894314,0.943458,0.502982


### Model 2: Ada Boost Classifier

In [124]:
#Ada Bost Classifier con datos escalados y sin la variable DepDelay
adab_3 = AdaBoostClassifier(n_estimators= 90, random_state=0)
adab_3.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=90, random_state=0)

In [125]:
metricas= obtener_metricas('AdaBoostClassifier_3', y_test, adab_3.predict(X_test))
metricas

{'Modelo': 'AdaBoostClassifier_3',
 'Accuracy': 0.8933312590748808,
 'Sensitivity': 0.9983947393869065,
 'Specificity': 0.009598177972994957,
 'Precision': 0.89450701784786,
 'F1Score': 0.9436000548370881,
 'ROC AUC Score': 0.5039964586799508}

In [126]:
#Añadimos las métricas al DF metricas2_sc para la comparativa posterior
metricas3 = metricas3.append(metricas, ignore_index=True)

### Model 3: BalancedBaggingClassifier

In [127]:
#BalancedBaggingClassifier con datos escalados y sin la variable DepDelay
bbc_3 = BalancedBaggingClassifier(n_estimators= 30, random_state=0)
bbc_3.fit(X_train, y_train)

BalancedBaggingClassifier(n_estimators=30, random_state=0)

In [128]:
metricas= obtener_metricas('BalancedBaggingClassifier_3', y_test, bbc_3.predict(X_test))
metricas

{'Modelo': 'BalancedBaggingClassifier_3',
 'Accuracy': 0.7112632233976354,
 'Sensitivity': 0.728633594429939,
 'Specificity': 0.5651537335285505,
 'Precision': 0.9337497211688601,
 'F1Score': 0.8185373484552209,
 'ROC AUC Score': 0.6468936639792447}

In [129]:
#Añadimos las métricas al DF metricas2_sc para la comparativa posterior
metricas3 = metricas3.append(metricas, ignore_index=True)

Comparamos las métricas de los modelos en los que no hemos utilizado la variable DepDelay con los modelos que si la utilizaron

In [132]:
pd.concat([metricas2, metricas3], axis=0).drop('F1Score_CV', axis=1)

,Modelo,Accuracy,Sensitivity,Specificity,Precision,F1Score,ROC AUC Score
0,LogisticRegression_optim,0.902804,0.980756,0.247112,0.916368,0.947470,0.613934
1,AdaBoostClassifier_optim,0.902873,0.974741,0.298357,0.921169,0.947198,0.636549
2,BalancedBaggingClassifier_optim,0.825175,0.822648,0.846429,0.978288,0.893743,0.834538
0,LogisticRegression_3,0.893055,0.998317,0.007646,0.894314,0.943458,0.502982
1,AdaBoostClassifier_3,0.893331,0.998395,0.009598,0.894507,0.943600,0.503996
2,BalancedBaggingClassifier_3,0.711263,0.728634,0.565154,0.933750,0.818537,0.646894


De los modelos que no utilizan la variable **DepDelay** observamos:
- Las métricas **Accuracy, Sensitivity y Precision** tienen unos valores similares a los modelos que utilizan DepDelay
- **ROC AUC Score** tambien presenta un valor más bajo, entre un 16% y un 22%
- La métrica más afectada es la **Specificity**. En el caso de LogisticRegression y AdaBoostClassifier pasa a ser 0 (practicamente no detecta los negativos). En el caso de BalancedBaggingClassifier la Specificity es de 0.56, un 33% menor

## Conclusiones finales

Trabajamos un data set con los vuelos en USA del 2008. El objetivo en encontrar un modelo que determine si el vuelo llegará retrasado o no. Características del dataset:
- Es un dataset **desbalanceado** (90% vuelos retrasados, 10% vuelos no retrasados)
- La variable **DepDelay** consigue una **accuracy y precision de 0.89** por ella sola, sin crear ningún modelo
- 5 variables (CarrierDelay, WeatherDelay, NASDelay, SecurityDelay,LateAircraftDelay suman el valor del target para vuelos que ArrDelay>15'
- El resto de variables presentan correlaciones muy bajas con el target
 
Planteamos 3 modelos de clasificación utilizando 9 variables, una de ellas dummificada. Uno de los modelos balancea los datos previamente y los otros 2 modelos los entrenamos con datos no balanceados y con datos balanceados. Concluímos:
- Los modelos no balanceados, aunque presentan una accuracy, sensitivity y precision altas, tienen en realidad un pobre desempeño ya que la specificity es muy baja (menor de 0.3)
- Al utilizar datos o modelos balanceados obtenemos una mejora sustancial en **specificity (más del doble)** y una ligera mejora en precisión. Por contra los modelos **bajan ligeramente en accuracy y moderardamente en sensitivity.**
- Los modelos con datos balanceados presentan una métricas parecidas, dependiendo de la metrica que deseemos optimizar eligiríamos modelo. 
- Ajustamos los parámetros de los modelos con GridSearch y conseguimos una **ligera mejora de todas las métricas (1,4%)**
- Entrenamos los modelos con Train/Test y con Cross Validation y **no observamos diferencias.** 

Planteamos los 3 modelos tras realizar una ingeniería de variables (algunas normalizamos y otras RobustScaler). No apreciamos diferencias significativas por lo que concluímos que **el escalado de variables no afecta** a los modelos estudiados

Finalmente creamos 3 modelos sin utilizar la variable DepDelay. Obtenemos:
- La **specificity** empeora significativamente (en el caso de los modelos que no utilizan datos balanceados llega a ser de 0)
- **ROC AUC Score** presenta un valor más bajo, entre un 16% y un 22%
- **Accuracy, Sensitivity y Precision** tienen unos valores similares a los modelos que utilizan DepDelay